In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import cv2
import tensorflow as tf
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


2023-10-03 04:04:42.451917: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-03 04:04:42.451958: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-03 04:04:42.451982: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-03 04:04:42.459586: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-03 04:04:43.585316: W tensorflow/compiler/

In [2]:
TRANSFORMS = transforms.Compose([
    transforms.ElasticTransform(alpha=125.0), 
    transforms.RandomRotation(15), 
    transforms.RandomHorizontalFlip()
    #maybe some more
])

In [19]:
data = pd.read_csv('./results.csv', sep='|')
print(data.shape)
data.head()

(158915, 3)


,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their...
1,1000092795.jpg,1,"Two young , White males are outside near many..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard .
3,1000092795.jpg,3,A man in a blue shirt standing in a garden .
4,1000092795.jpg,4,Two friends enjoy time spent together .


In [20]:
# column names have spaces in the beginning for some reason
# (plolly cause the seperator is '|' and not space)
data.rename(columns={' comment_number': 'comment_number',
            ' comment': 'comment'}, inplace=True)
# there was one empty comment
data.dropna(inplace=True)

In [22]:
import re
def remove_non_alpha(s: str):
    s = s.lower()
    s = re.sub(r'[^\w\s]', '', s)
    return s

In [24]:
data['comment'] = data['comment'].apply(remove_non_alpha)
data['comment'].values

array([' two young guys with shaggy hair look at their hands while hanging out in the yard ',
       ' two young  white males are outside near many bushes ',
       ' two men in green shirts are standing in a yard ', ...,
       ' a man is leaning off of the side of a blue and white boat as it sits in a body of water ',
       ' a man riding a small boat in a harbor  with fog and mountains in the background ',
       ' a man on a moored blue and white boat with hills and mist in the background '],
      dtype=object)

In [25]:
# forming a dictionary for the model
from collections import Counter

words = Counter()
for sent in data['comment'].values:
    for word in sent.split():
        words[word] +=1


[('a', 271704), ('in', 83466), ('the', 62978), ('on', 45669), ('and', 44263), ('man', 42598), ('is', 41117), ('of', 38776), ('with', 36207), ('woman', 22211), ('two', 21642), ('are', 20196), ('to', 17607), ('people', 17337), ('at', 16259), ('an', 15883), ('wearing', 15709), ('young', 13218), ('white', 13178), ('shirt', 12981), ('black', 12311), ('while', 11711), ('his', 11495), ('blue', 11310), ('red', 9916), ('girl', 9655), ('sitting', 9620), ('men', 9498), ('boy', 9429), ('standing', 9113), ('dog', 9089), ('playing', 8620), ('street', 7998), ('group', 7852), ('down', 7678), ('front', 7604), ('her', 7414), ('walking', 7339), ('holding', 6985), ('one', 6484), ('water', 5957), ('by', 5771), ('three', 5770), ('women', 5275), ('green', 5221), ('up', 5114), ('child', 5078), ('looking', 5067), ('as', 5028), ('for', 4899), ('little', 4794), ('large', 4753), ('outside', 4702), ('yellow', 4557), ('person', 4479), ('children', 4454), ('brown', 4446), ('through', 4297), ('from', 4041), ('their',

In [26]:
print(len(words))

20007


In [31]:
MAX_DICTIONARY_LENGTH = 15000
most_common = words.most_common(MAX_DICTIONARY_LENGTH)
# is the threshold needed?
dictionary = pd.DataFrame(
    {'word': [i[0] for i in most_common if i[1] >= 10], 'count': [i[1] for i in most_common if i[1] >= 10]})

In [32]:
dictionary.shape

(5461, 2)

In [ ]:
class my_lstm(nn.Module):
    # gotta get rid of this later, the experiment was fine enough
    def __init__(self, input_size, hidden_size,  *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.hidden = torch.rand(hidden_size)
        self.cell_state = torch.rand(hidden_size)

        self.forget_gate = nn.Sequential(
            nn.Linear(hidden_size+input_size, hidden_size),
            nn.Sigmoid()
        )
        self.input_gate = nn.Sequential(
            nn.Linear(hidden_size+input_size, hidden_size),
            nn.Sigmoid()
        )
        self.cell_gate = nn.Sequential(
            nn.Linear(hidden_size+input_size, hidden_size),
            nn.Tanh()
        )
        self.output_gate = nn.Sequential(
            nn.Linear(hidden_size+input_size, hidden_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        hx = torch.concat((x, self.hidden))
        self.cell_state *= self.forget_gate(hx)
        self.cell_state += self.cell_gate(hx)*self.input_gate(hx)
        out = torch.tanh(self.cell_state) * self.output_gate(hx)
        self.hidden = out
        return out

In [1]:
import tensorflow as tf

2023-10-03 04:07:51.284682: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-03 04:07:51.284724: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-03 04:07:51.284748: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-03 04:07:51.291785: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-03 04:07:52.419590: W tensorflow/compiler/

In [2]:
tf.config.list_physical_devices('GPU')

2023-10-03 04:10:21.473967: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-03 04:10:21.484535: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-03 04:10:21.484787: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]